### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = ".//Resources/schools_complete.csv"
student_data_to_load = ".//Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
schools_data = pd.read_csv(school_data_to_load)
students_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
merge_df = pd.merge(students_data, schools_data, how="left", on=["school_name", "school_name"])


In [2]:
##prep data- hold them in variables
#hold unique names in a dataframe
schools = schools_data['school_name'].unique()

#counts each unique school to return total school
school_count = len(schools)

#counts number of students
students = schools_data['size'].sum()

#student count from student file (to verify with district student count)
#test_student = students_data['student_name'].count()

#total budget
total_budget = schools_data['budget'].sum()

#save pct for formatting- *100 gives like 6 decimals- yuck
#passing and % pass reading
read_pass = students_data.loc[students_data['reading_score'] >= 60]['reading_score'].count()
read_pass_pct = read_pass/students

#passing and % pass math
math_pass = students_data.loc[students_data['math_score'] >= 60]['math_score'].count()
math_pass_pct = math_pass/students

#average math score
avg_math = students_data['math_score'].mean()

#average reading score
avg_reading = students_data['reading_score'].mean()

#Overall Passing Rate
both_pass = students_data[(students_data['math_score'] >= 60) & (students_data['reading_score'] >= 60)]['student_name'].count()/students





## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
#create a dataframe and print it out
# district dataframe from dictionary

district_summary = pd.DataFrame({
    
    "Total Schools": [school_count],
    "Total Students": [students],
    "Total Budget": [total_budget],
    "Average Reading Score": [avg_reading],
    "Average Math Score": [avg_math],
    "% Passing Reading":[read_pass_pct],
    "% Passing Math": [math_pass_pct],
    "Overall Passing Rate": [both_pass]

})

#store as different df to change order
dist_sum = district_summary[[
                             "Total Schools", 
                             "Total Students", 
                             "Total Budget", 
                             "Average Reading Score", 
                             "Average Math Score", 
                             '% Passing Reading', 
                             '% Passing Math', 
                             'Overall Passing Rate'
                            ]]

#beautification of data-
dist_sum.style.format({"Total Budget": "${:,.2f}", 
                       "Average Reading Score": "{:.1f}", 
                       "Average Math Score": "{:.1f}", 
                       "% Passing Math": "{:.1%}", 
                       "% Passing Reading": "{:.1%}", 
                       "Overall Passing Rate": "{:.1%}"})


,Total Schools,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
0,15,39170,"$24,649,428.00",81.9,79.0,100.0%,92.4%,92.4%


## School Summary


* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [5]:

#summarize based on each school
school = merge_df.set_index('school_name').groupby(['school_name'])

#summarization on types of schools
types = schools_data.set_index('school_name')['type']

# total student count in each school
student_count = school['Student ID'].count()

# school budget
budget = schools_data.set_index('school_name')['budget']

#budget for each student
student_budget = schools_data.set_index('school_name')['budget']/schools_data.set_index('school_name')['size']

#averages for math and reading across schools
avg_math = school['math_score'].mean()
avg_read = school['reading_score'].mean()


# % passing scores
#format to fix the % and set decimal places!
pass_math = merge_df[merge_df['math_score'] >= 70].groupby('school_name')['Student ID'].count()/student_count
pass_read = merge_df[merge_df['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/student_count 
overall = merge_df[(merge_df['reading_score'] >= 70) & (merge_df['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/student_count

schsummary = pd.DataFrame({
    "School Type": types,
    "Total Students": budget,
    "Per Student Budget": student_budget,
    "Total School Budget": budget,
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    "% Passing Math": pass_math,
    "% Passing Reading": pass_read,
    "Overall Passing Rate": overall
})

#store as different df to change order
school_sum = schsummary[[
                             "School Type", 
                             "Total Students", 
                             "Per Student Budget", 
                             "Total School Budget", 
                             "Average Math Score",
                             "Average Reading Score",
                             "% Passing Reading", 
                             "% Passing Math", 
                             "Overall Passing Rate"
                        ]]

#format cells
#budgets --$ formatting
#totals -- comma formatting
#%-- pct formatting
school_sum.style.format({
                       "Total Budget": "${:,.2f}",
                       "Total Students": "{:,}",
                       "Average Reading Score": "{:.1f}", 
                       "Average Math Score": "{:.1f}", 
                       "Total School Budget":"${:,.2f}",
                       "Per Student Budget": "${:,.2f}",
                       "% Passing Math": "{:.1%}", 
                       "% Passing Reading": "{:.1%}", 
                       "Overall Passing Rate": "{:.1%}"
                       })


,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Reading,% Passing Math,Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,"3,124,928",$628.00,"$3,124,928.00",77.0,81.0,81.9%,66.7%,54.6%
Cabrera High School,Charter,"1,081,356",$582.00,"$1,081,356.00",83.1,84.0,97.0%,94.1%,91.3%
Figueroa High School,District,"1,884,411",$639.00,"$1,884,411.00",76.7,81.2,80.7%,66.0%,53.2%
Ford High School,District,"1,763,916",$644.00,"$1,763,916.00",77.1,80.7,79.3%,68.3%,54.3%
Griffin High School,Charter,"917,500",$625.00,"$917,500.00",83.4,83.8,97.1%,93.4%,90.6%
Hernandez High School,District,"3,022,020",$652.00,"$3,022,020.00",77.3,80.9,80.9%,66.8%,53.5%
Holden High School,Charter,"248,087",$581.00,"$248,087.00",83.8,83.8,96.3%,92.5%,89.2%
Huang High School,District,"1,910,635",$655.00,"$1,910,635.00",76.6,81.2,81.3%,65.7%,53.5%
Johnson High School,District,"3,094,650",$650.00,"$3,094,650.00",77.1,81.0,81.2%,66.1%,53.5%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [6]:
#top schools by overall passing percentages
best = school_sum.sort_values("Overall Passing Rate", ascending = False)
#format for readability
best.head().style.format({
                           "Total Students": '{:,}',
                           "Total School Budget": "${:,}", 
                           "Per Student Budget": "${:.0f}", 
                           "% Passing Math": "{:.1%}", 
                           "% Passing Reading": "{:.1%}", 
                           "Overall Passing Rate": "{:.1%}"})

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Reading,% Passing Math,Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,"1,081,356",$582,"$1,081,356",83.061895,83.975780,97.0%,94.1%,91.3%
Thomas High School,Charter,"1,043,130",$638,"$1,043,130",83.418349,83.848930,97.3%,93.3%,90.9%
Griffin High School,Charter,"917,500",$625,"$917,500",83.351499,83.816757,97.1%,93.4%,90.6%
Wilson High School,Charter,"1,319,574",$578,"$1,319,574",83.274201,83.989488,96.5%,93.9%,90.6%
Pena High School,Charter,"585,858",$609,"$585,858",83.839917,84.044699,95.9%,94.6%,90.5%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [7]:

#reverse results from best to return (ascending=true)
worst = school_sum.sort_values("Overall Passing Rate", ascending = True)
#format for readability
worst.head().style.format({
                           "Total Students": '{:,}',
                           "Total School Budget": "${:,}", 
                           "Per Student Budget": "${:.0f}", 
                           "% Passing Math": "{:.1%}", 
                           "% Passing Reading": "{:.1%}", 
                           "Overall Passing Rate": "{:.1%}"})

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Reading,% Passing Math,Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,"2,547,363",$637,"$2,547,363",76.842711,80.744686,80.2%,66.4%,53.0%
Figueroa High School,District,"1,884,411",$639,"$1,884,411",76.711767,81.158020,80.7%,66.0%,53.2%
Huang High School,District,"1,910,635",$655,"$1,910,635",76.629414,81.182722,81.3%,65.7%,53.5%
Hernandez High School,District,"3,022,020",$652,"$3,022,020",77.289752,80.934412,80.9%,66.8%,53.5%
Johnson High School,District,"3,094,650",$650,"$3,094,650",77.072464,80.966394,81.2%,66.1%,53.5%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [8]:
#separate mathscore averages for each school year into variables
math_fresh = students_data.loc[students_data['grade'] == '9th'].groupby('school_name')["math_score"].mean()
math_soph = students_data.loc[students_data['grade'] == '10th'].groupby('school_name')["math_score"].mean()
math_jun = students_data.loc[students_data['grade'] == '11th'].groupby('school_name')["math_score"].mean()
math_sen = students_data.loc[students_data['grade'] == '12th'].groupby('school_name')["math_score"].mean()

#create dataframe using these variables
math_scores = pd.DataFrame({
        "9th": math_fresh,
        "10th": math_soph,
        "11th": math_jun,
        "12th": math_sen
})
#create list and index by school_name
math_scores = math_scores[['9th', '10th', '11th', '12th']]
math_scores.index.name = "School"

#show and format
math_scores.style.format({"9th": '{:.1f}', 
                          "10th": '{:.1f}', 
                          "11th": "{:.1f}", 
                          "12th": "{:.1f}"})


,9th,10th,11th,12th
School,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [9]:
#separate reading averages for each school year into variables--just copy/paste above and swap names to reading references
read_fresh = students_data.loc[students_data['grade'] == '9th'].groupby('school_name')["reading_score"].mean()
read_soph = students_data.loc[students_data['grade'] == '10th'].groupby('school_name')["reading_score"].mean()
read_jun = students_data.loc[students_data['grade'] == '11th'].groupby('school_name')["reading_score"].mean()
read_sen = students_data.loc[students_data['grade'] == '12th'].groupby('school_name')["reading_score"].mean()

#create dataframe using these variables
read_scores = pd.DataFrame({
        "9th": read_fresh,
        "10th": read_soph,
        "11th": read_jun,
        "12th": read_sen
})
#create list and index by school_name
read_scores = read_scores[['9th', '10th', '11th', '12th']]
read_scores.index.name = "School"

#show and format
read_scores.style.format({'9th': '{:.1f}', 
                          "10th": '{:.1f}', 
                          "11th": "{:.1f}", 
                          "12th": "{:.1f}"})

,9th,10th,11th,12th
School,,,,
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [10]:
# create bins for school budget
bins = [0, 584.999, 614.999, 644.999, 999999]
labels = ['< $585', "$585 - 614", "$615 - 644", "> $644"]
merge_df['spend_bins'] = pd.cut(merge_df['budget']/merge_df['size'], bins, labels = labels)

#group by spending
spending = merge_df.groupby('spend_bins')

#average for math/reading, pct pass for math/reading and both
##DO NOT PANIC!!! there are a lot of 100% results because my 'passing score' is set to be greater than or = 60
avg_math = spending['math_score'].mean()
avg_read = spending['reading_score'].mean()
pass_math = merge_df[merge_df['math_score'] >= 60].groupby('spend_bins')['Student ID'].count()/spending['Student ID'].count()
pass_read = merge_df[merge_df['reading_score'] >= 60].groupby('spend_bins')['Student ID'].count()/spending['Student ID'].count()
overall = merge_df[(merge_df['reading_score'] >= 60) & (merge_df['math_score'] >= 60)].groupby('spend_bins')['Student ID'].count()/spending['Student ID'].count()

            
# df build            
spend = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
            
                    })
            
#reorder columns
spend = spend[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
            ]]

spend.index.name = "Per Student Budget"
spend = spend.reindex(labels)

#format columns
spend.style.format({
                              "Average Math Score": '{:.1f}', 
                              "Average Reading Score": '{:.1f}', 
                              "% Passing Math": '{:.1%}', 
                              "% Passing Reading":'{:.1%}', 
                              "Overall Passing Rate": '{:.1%}'
                    })

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Per Student Budget,,,,,
< $585,83.4,84.0,100.0%,100.0%,100.0%
$585 - 614,83.5,83.8,100.0%,100.0%,100.0%
$615 - 644,78.1,81.4,90.9%,100.0%,90.9%
> $644,77.0,81.0,89.1%,100.0%,89.1%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [11]:
# create size bins
bins = [0, 999, 1999, 99999999999]
labels = ["Small (<1000)", "Medium (1000-2000)" , "Large (>2000)"]
merge_df['sizes'] = pd.cut(merge_df['size'], bins, labels = labels)

#group by size
size = merge_df.groupby('sizes')

#averages and percentages
#PANIC! no, wait...settle down, my scores for 'passing' are above 59.9%, so i have high percentage of passing
avg_math = size['math_score'].mean()
avg_read = size['math_score'].mean()
pass_math = merge_df[merge_df['math_score'] >= 60].groupby('sizes')['Student ID'].count()/size['Student ID'].count()
pass_read = merge_df[merge_df['reading_score'] >= 60].groupby('sizes')['Student ID'].count()/size['Student ID'].count()
overall = merge_df[(merge_df['reading_score'] >= 60) & (merge_df['math_score'] >= 60)].groupby('sizes')['Student ID'].count()/size['Student ID'].count()

            
# df build            
size = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    "% Passing Math": pass_math,
    "% Passing Reading": pass_read,
    "Overall Passing Rate": overall
            
})
            
#reorder columns
size = size[[
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "Overall Passing Rate"
            ]]

size.index.name = "Total Students"
size = size.reindex(labels)

#formating
size.style.format({
                              "Average Math Score": '{:.1f}', 
                              "Average Reading Score": '{:.1f}', 
                              "% Passing Math": '{:.1%}', 
                              "% Passing Reading":'{:.1%}', 
                              "Overall Passing Rate": '{:.1%}'
                })

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Total Students,,,,,
Small (<1000),83.8,83.8,100.0%,100.0%,100.0%
Medium (1000-2000),83.4,83.4,100.0%,100.0%,100.0%
Large (>2000),77.5,77.5,89.9%,100.0%,89.9%


## Scores by School Type

* Perform the same operations as above, based on school type

In [12]:
# group by type of school
type = merge_df.groupby("type")

#avg and pct for math/read/overall- panic! several 100%, unable to locate a definitive pass score for schools- used 60 
avg_math = type['math_score'].mean()
avg_read = type['math_score'].mean()
pass_math = merge_df[merge_df['math_score'] >= 60].groupby('type')['Student ID'].count()/type['Student ID'].count()
pass_read = merge_df[merge_df['reading_score'] >= 60].groupby('type')['Student ID'].count()/type['Student ID'].count()
overall = merge_df[(merge_df['reading_score'] >= 60) & (merge_df['math_score'] >= 60)].groupby('type')['Student ID'].count()/type['Student ID'].count()

# df build            
type = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    "% Passing Math": pass_math,
    "% Passing Reading": pass_read,
    "Overall Passing Rate": overall})
    
#reorder columns
type = type[[
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "Overall Passing Rate"
            ]]
type.index.name = "Type of School"


#formating
type.style.format({
                              "Average Math Score": '{:.1f}', 
                              "Average Reading Score": '{:.1f}', 
                              "% Passing Math": '{:.1%}', 
                              "% Passing Reading":'{:.1%}', 
                              "Overall Passing Rate": '{:.1%}'
                })

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Type of School,,,,,
Charter,83.4,83.4,100.0%,100.0%,100.0%
District,77.0,77.0,89.0%,100.0%,89.0%
